In [60]:
import pandas as pd
import glob
import altair as alt
from sublimpy import utils
import datetime as dt
import numpy as np

In [61]:
files = glob.glob("../turb_datasets/tidy_df**.parquet")
df = pd.DataFrame()
for file in files:
    local = pd.read_parquet(file).query("measurement == 'w_h2o_'")
    df = pd.concat([df, local.assign(filename = file)])
df = df[df.variable.isin([
    'w_h2o__2m_c', 'w_h2o__3m_c', 'w_h2o__5m_c', 'w_h2o__10m_c', 'w_h2o__15m_c', 'w_h2o__20m_c', 
    'w_h2o__3m_uw', 'w_h2o__10m_uw', 
    'w_h2o__3m_ue', 'w_h2o__10m_ue', 
    'w_h2o__3m_d', 'w_h2o__10m_d', 
])]
df = utils.modify_df_timezone(df, 'UTC', 'US/Mountain')
df = df[(df['time'] > '20221130') & (df['time'] < '20230508')]

In [62]:
df['filename'] = df['filename'].str.replace('turb_datasets/tidy_df_20221101_20230619_planar_fit_multiplane_STRAIGHTUP_', '')
df['processing'] = df['filename'].str.replace('.parquet', '')

In [63]:
df.head()

,time,variable,value,height,tower,measurement,filename,processing
3993087,2022-11-30 00:30:00,w_h2o__3m_uw,-0.000622,3.0,uw,w_h2o_,../nodespiking_flags36000_snowfallfiltered0.5m...,../nodespiking_flags36000_snowfallfiltered0.5mm
3993088,2022-11-30 01:00:00,w_h2o__3m_uw,-0.000043,3.0,uw,w_h2o_,../nodespiking_flags36000_snowfallfiltered0.5m...,../nodespiking_flags36000_snowfallfiltered0.5mm
3993089,2022-11-30 01:30:00,w_h2o__3m_uw,-0.000308,3.0,uw,w_h2o_,../nodespiking_flags36000_snowfallfiltered0.5m...,../nodespiking_flags36000_snowfallfiltered0.5mm
3993090,2022-11-30 02:00:00,w_h2o__3m_uw,-0.001253,3.0,uw,w_h2o_,../nodespiking_flags36000_snowfallfiltered0.5m...,../nodespiking_flags36000_snowfallfiltered0.5mm
3993091,2022-11-30 02:30:00,w_h2o__3m_uw,0.000094,3.0,uw,w_h2o_,../nodespiking_flags36000_snowfallfiltered0.5m...,../nodespiking_flags36000_snowfallfiltered0.5mm


# Apply plausibility limit

In [70]:
df = df[np.abs(df.value) < 0.2]

# Calculate and plot seasonal totals

In [72]:
totals_df = pd.DataFrame(df.groupby(['variable', 'processing'])['value'].sum()*30*60/1000).reset_index()

In [73]:
totals_df['despiking'] = totals_df['processing'].apply(lambda s: s.split('_')[0])
totals_df['instrument_flagging'] = totals_df['processing'].apply(lambda s: s.split('_')[1])
totals_df['snowfall_filtering'] = totals_df['processing'].apply(lambda s: s.split('_')[2])
totals_df['height'] = totals_df['variable'].str.split('_').str[3].str[:-1].astype(int)
totals_df['tower'] = totals_df['variable'].str.split('_').str[-1]
totals_df

,variable,processing,value,despiking,instrument_flagging,snowfall_filtering,height,tower
0,w_h2o__10m_c,../nodespiking_flags36000_snowfallfiltered0.5mm,37.811346,../nodespiking,flags36000,snowfallfiltered0.5mm,10,c
1,w_h2o__10m_c,../nodespiking_flags36000_snowfallfiltered0mm,36.147612,../nodespiking,flags36000,snowfallfiltered0mm,10,c
2,w_h2o__10m_c,../nodespiking_flags36000_snowfallfilteredno,36.201793,../nodespiking,flags36000,snowfallfilteredno,10,c
3,w_h2o__10m_c,../nodespiking_flags3600_snowfallfiltered0.5mm,35.906514,../nodespiking,flags3600,snowfallfiltered0.5mm,10,c
4,w_h2o__10m_c,../nodespiking_flags3600_snowfallfiltered0mm,34.431823,../nodespiking,flags3600,snowfallfiltered0mm,10,c
...,...,...,...,...,...,...,...,...
211,w_h2o__5m_c,../q7_flags3600_snowfallfiltered0mm,32.703641,../q7,flags3600,snowfallfiltered0mm,5,c
212,w_h2o__5m_c,../q7_flags3600_snowfallfilteredno,34.031976,../q7,flags3600,snowfallfilteredno,5,c
213,w_h2o__5m_c,../q7_flags9000_snowfallfiltered0.5mm,34.422582,../q7,flags9000,snowfallfiltered0.5mm,5,c
214,w_h2o__5m_c,../q7_flags9000_snowfallfiltered0mm,33.021581,../q7,flags9000,snowfallfiltered0mm,5,c


In [74]:
alt.Chart(totals_df).mark_point(size=100).encode(
    alt.X('value:Q').scale(zero=False, domain=[20,40]),
    alt.Y('height:Q'),
    alt.Shape('tower:N'),
    alt.Column('instrument_flagging').sort(['flags36000', 'flags9000', 'flags3600']),
    alt.Row('despiking'),
    alt.Color('snowfall_filtering')
).properties(width=150, height = 150)

alt.Chart(...)

In [76]:
base_chart = alt.Chart(totals_df).encode(
    alt.Y('variable:O'),
).properties(width=150, height = 150)

base_chart.mark_rule().encode(
    alt.X('min(value):Q').scale(zero=False),
    alt.X2('max(value):Q'),
) + base_chart.mark_circle().encode(
    alt.X('mean(value):Q'),
)

alt.LayerChart(...)

In [78]:
round(100 * (
    totals_df.groupby('variable')['value'].max() 
    - totals_df.groupby('variable')['value'].min()
) / totals_df.groupby('variable')['value'].min(), 1)

variable
w_h2o__10m_c     15.3
w_h2o__10m_d     11.4
w_h2o__10m_ue    11.9
w_h2o__10m_uw    10.2
w_h2o__15m_c     27.1
w_h2o__20m_c     18.3
w_h2o__2m_c       7.5
w_h2o__3m_c      10.8
w_h2o__3m_d       6.4
w_h2o__3m_ue     11.2
w_h2o__3m_uw      8.8
w_h2o__5m_c      11.9
Name: value, dtype: float64

# Calculate and examine seasonal cumulative

In [79]:
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [80]:
VARIABLE = 'w_h2o__20m_c'
processing_options = df.processing.unique()
local = pd.DataFrame()
for process in processing_options:
    local = pd.concat([
        local,
        df.query(f"variable == '{VARIABLE}'").query(f"processing == '{process}'").set_index('time').sort_index()[['value']].cumsum().assign(processing = process)
    ])
local['value'] = local['value']*1.8
local['despiking'] = local['processing'].apply(lambda s: s.split('_')[0])
local['instrument_flagging'] = local['processing'].apply(lambda s: s.split('_')[1])
local['snowfall_filtering'] = local['processing'].apply(lambda s: s.split('_')[2])

In [81]:
alt.Chart(local.reset_index().dropna()).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('processing:N'),
    alt.Row('snowfall_filtering:N'),
    tooltip = 'processing:N'
).properties(width=1000).resolve_scale(color='independent')

alt.Chart(...)

In [ ]:
pd.options.display.max_rows = 500
display(df.groupby(['variable','processing']).value.max())

In [ ]:
def restrict_time(src):
    return src[
        (
            src.time > '20230111'
        )&(
            src.time < '20230111 0600'
        )
    ]
alt.Chart(restrict_time(
    df.query("variable == 'w_h2o__20m_c'")
)).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('processing'),
    facet='processing'
)

In [ ]:
def restrict_time(src):
    return src[
        (
            src.time > '20221221'
        )&(
            src.time < '20221223'
        )
    ]
alt.Chart(restrict_time(
    df.query("variable == 'w_h2o__20m_c'")
)).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('processing'),
    facet='processing'
)

In [ ]:
restrict_time(
    df.query("variable == 'w_h2o__20m_c'")
).value.value_counts()

# Apply mean diurnal gap filling 

In [ ]:
df_gapfilled = pd.DataFrame()
for lhflux_variable in df.variable.unique():
    for process in df.processing.unique():
        subset = df.query(f"variable == '{lhflux_variable}'")
        subset = subset.query(f"processing == '{process}'")
        subset = subset.set_index('time')
        for i,row in subset.iterrows():
            if np.isnan(row['value']) or row['value'] == 0:
                start_window = i - dt.timedelta(days=3, hours=12)
                end_window = i + dt.timedelta(days=3, hours=12)
                src = subset.loc[start_window: end_window].reset_index()
                means = pd.DataFrame(
                    src.groupby([src.time.dt.hour, src.time.dt.minute])['value'].mean()
                )
                subset.loc[i, 'value'] = means.loc[i.hour, i.minute].value
        df_gapfilled = pd.concat([df_gapfilled, subset])

In [ ]:
totals_gapfilled_df = pd.DataFrame(df_gapfilled.groupby(['variable', 'processing'])['value'].sum()*30*60/1000).reset_index()

In [ ]:
totals_gapfilled_df['despiking'] = totals_gapfilled_df['processing'].apply(lambda s: s.split('_')[0])
totals_gapfilled_df['instrument_flagging'] = totals_gapfilled_df['processing'].apply(lambda s: s.split('_')[1])
totals_gapfilled_df['snowfall_filtering'] = totals_gapfilled_df['processing'].apply(lambda s: s.split('_')[2])
totals_gapfilled_df['height'] = totals_gapfilled_df['variable'].str.split('_').str[3].str[:-1].astype(int)
totals_gapfilled_df['tower'] = totals_gapfilled_df['variable'].str.split('_').str[-1]
totals_gapfilled_df

In [ ]:
alt.Chart(totals_gapfilled_df).mark_point(size=100).encode(
    alt.X('value:Q').scale(zero=False, domain=[20,40]),
    alt.Y('height:Q'),
    alt.Shape('tower:N'),
    alt.Column('instrument_flagging').sort(['flags36000', 'flags9000', 'flags3600']),
    alt.Row('despiking'),
    alt.Color('snowfall_filtering')
).properties(width=150, height = 150).display(renderer='svg')

In [ ]:
def restrict_time(src):
    return src[
        (
            src.time > '20221221'
        )&(
            src.time < '20221223'
        )
    ]
alt.Chart(restrict_time(
    df.query("variable == 'w_h2o__20m_c'")
)).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('processing'),
    facet='processing'
)